In [25]:
import ps
import ps_utils
import numpy as np
import pandas as pd
import datetime
#ps_utils.switch_to_prod_pvlss()
#ps_utils.switch_to_prod_sigma()
ps_utils.switch_to_dev()
#ps_utils.switch_to_psi()
import matplotlib.pyplot as plt
#ps_utils.switch_to_psi()
dupire_model='DupireLocalVolatilityModel'
no_skew_model = 'HybridLGM1FDupireLocalVolatilityModel'
skew_model = 'HybridCheyette1FactorLocalVolatilityModel'
tarn_model = 'Cheyette1FactorModelMC'
mc_model = 'HybridCheyette1FactorMCLocalVolatilityModel'
an_model = 'HybridCheyette1FactorANLocalVolatilityModel'

float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

print(ps_utils.global_algo_id())

print(ps_utils.__file__)
print(ps_utils.get_env_url('psi'))

# tek=ps_utils.pricing.XVA(ps.get('545ed88abf05761382a40cf175c503f700d8bd59'))
# prev=ps_utils.pricing.XVA(ps.get('5794aa54598727d3b90024f981d88ce272d29054'))
# ps_utils.run_generic_revaluation_based_metric_change_explain(['BCVA'],'USD',tek, prev)

# lambd=(100+319)/10000/(1-0.25)
# #lambd=np.log((1-0.25)/(1-0.25-(100+1655)/10000))
# #N=45.4*1e6/r.Model.MarketDataSet.Spots['USDRUB']
# N1=sum(r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums)/r.Model.MarketDataSet.Spots['USDRUB']
# N2=(sum(r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums) +221000)/r.Model.MarketDataSet.Spots['USDRUB']
# print("N1=",N1,"N2=",N2)
# T1=(datetime.datetime(2023,7,28)-datetime.datetime.now()).days/365
# T2=(datetime.datetime(2023,10,30)-datetime.datetime.now()).days/365
# (1-0.25)*(N2*(1-np.exp(-lambd*T2))- N1*(1-np.exp(-lambd*T1)))

[INFO] 2023-11-09 12:22:38,558 - Your ps_utils version is 23.1.1, it is up to date
[WARNING] 2023-11-09 12:22:38,562 - You do not have xbbg installed, some functionality will not be available
[INFO] 2023-11-09 12:22:41,487 - Nothing to do: current environment is already dev


4378aad7661ecd69fcd1b35066a0aa8da52739a5
/Users/19865272/Documents/ПрограммыПитон/gitrepositoriesMAC_strat/strat/src/ps_utils/__init__.py
http://tkles-dealm0004.vm.esrt.cloud.sbrf.ru


In [26]:
def get_bcva_df(a):
    print('PV=',a['ProductValue'])
    cva_vals=[]
    cva_mas1=[]
    cva_mas2=[]
    dva_vals=[]
    dva_mas1=[]
    dva_mas2=[]
    for el in a:
        if el.startswith('CVA'):
            if type(a[el])!=ps.message.Dict:
                cva_mas1.append(el[1:])
                cva_mas2.append(el[1:])
                cva_vals.append(a[el])
            if type(a[el])==ps.message.Dict:
                for k in a[el]:
                    cva_mas1.append(el[4:])
                    cva_mas2.append(k)
                    if (el.endswith('Exposure')):
                        cva_vals.append(a[el][k])
                    elif (el.endswith('CreditDelta') or el.endswith('Rho') or el.endswith('RatesVega')):
                        cva_vals.append(a[el][k]/10000)
                    elif (el.endswith('ExchangeVega')):
                        cva_vals.append(a[el][k]/100)
                    else:
                        print('Unexpected item:',el)
                        #cva_vals.append(a[el][k])
        elif el.startswith('DVA'):
            if type(a[el])!=ps.message.Dict:
                dva_mas1.append(el[1:])
                dva_mas2.append(el[1:])
                dva_vals.append(a[el])
            if type(a[el])==ps.message.Dict:
                for k in a[el]:
                    dva_mas1.append(el[4:])
                    dva_mas2.append(k)
                    if (el.endswith('Exposure')):
                        dva_vals.append(a[el][k])
                    elif (el.endswith('CreditDelta') or el.endswith('Rho') or el.endswith('RatesVega')):
                        dva_vals.append(a[el][k]/10000)
                    elif (el.endswith('ExchangeVega')):
                        dva_vals.append(a[el][k]/100)
                    else:
                        print('Unexpected item:',el)
                        #dva_vals.append(a[el][k])
        else:
            print('Unexpected item:',el)
                        
    cva_arrays = [np.array(cva_mas1),np.array(cva_mas2)]
    cva_df=pd.DataFrame(cva_vals, index=cva_arrays)
    dva_arrays = [np.array(dva_mas1),np.array(dva_mas2)]
    dva_df=pd.DataFrame(dva_vals, index=dva_arrays)
   
    vals=np.zeros(len(cva_df.values)*3).reshape(-1,3)
    vals[:,0]=cva_df.values.flatten()
    vals[:,1]=dva_df.values.flatten()
    vals[:,2]=(cva_df.values+dva_df.values).flatten()
    
    
    otv=pd.DataFrame(vals, index=cva_df.index)
    otv=otv.rename(columns={0:'CVA',1:'DVA',2:'BCVA'})
    return otv


def get_pv_df(a):
    print('PV=',a['ProductValue'])
    pv_vals=[]
    pv_mas1=[]
    pv_mas2=[]
    for el in a:
        if type(a[el])!=ps.message.Dict and el not in ['StandardError','ReportingCurrency','CashToSettleToday','BCVA']:
            print(el)
            pv_mas1.append(el[1:])
            pv_mas2.append(el[1:])
            pv_vals.append(a[el])
        if type(a[el])==ps.message.Dict:
            for k in a[el]:
                pv_mas1.append(el[4:])
                pv_mas2.append(k)
                if (el.endswith('Exposure')):
                    pv_vals.append(a[el][k])
                elif (el.endswith('CreditDelta') or el.endswith('Rho') or el.endswith('RatesVega')):
                    pv_vals.append(a[el][k]/10000)
                elif (el.endswith('ExchangeVega')):
                    pv_vals.append(a[el][k]/100)
                else:
                    print('Unexpected item:',el)
                    
    
    pv_arrays = [np.array(pv_mas1),np.array(pv_mas2)]
    pv_df=pd.DataFrame(pv_vals, index=pv_arrays)
  
    
    vals=np.zeros(len(pv_df.values)*1).reshape(-1,1)
    vals[:,0]=pv_df.values.flatten()
   
    otv=pd.DataFrame(vals, index=pv_df.index)
    otv=otv.rename(columns={0:'PV'})
    return otv

def get_tenored_dv01(r,ircurvename,algo_id,use_aad=True, use_bump=False, use_bump_updown=False,
                     only_pv_greeks=False, is_batch=False):
    if ircurvename not in ['RUB_KEY_RATE','RUB_RUONIA_OIS']:
            raise Exception("Only RUB_KEY_RATE and RUB_RUONIA_OIS DV01 are supported")
    r=ps.get(r) 
    if not is_batch:
        if r.Model.TypeName in ['Cheyette1FactorModelMC','HybridCheyette1FactorMCLocalVolatilityModel']:
            if 'NettingSets' in r.Product:
                r.Product=r.Product.NettingSets[0].MarginSets[0].Products[0]
            r.Model.LocalVolatilityMinimum=1e-13
            r.Model.CalibrationFrequencyInDays=30
            r.Model.LocalVolNumberOfTimeStepsPerYear=250
            r.Model.MonteCarloMaxStepInHours=24
        if use_aad:
            r.Queries=["Rho",'CVA_Rho','DVA_Rho']
        if only_pv_greeks:
            r.Queries.remove('CVA_Rho')
            r.Queries.remove('DVA_Rho')
        if use_bump:
            r.Queries.append('RhoBump')
        if use_bump_updown:
            r.Queries.append('RhoBumpUpDown')
        z=ps_utils.compute(ps.put(r),algo_id=algo_id)
        dates=list(map(lambda x: x.date(),r.Model.MarketDataSet.RatesCurvesBundles[ircurvename].RatesSchedule.Dates))
        tenors=list(map(lambda x: (x-datetime.datetime.now().date()).days/365, dates))
        column_names=['Dates','Tenors']
        mas_of_column_values=[dates,tenors]
        if 'Rho' in z:
            column_names.append('PV_Rho')
            mas_of_column_values.append(np.array(z['Rho'][ircurvename])/10000)
        if 'RhoBump' in z:
            column_names.append('PV_RhoBump')
            mas_of_column_values.append(np.array(z['RhoBump'][ircurvename])/10000)
        if 'RhoBumpUpDown' in z:
            column_names.append('PV_RhoBumpUpDown')
            mas_of_column_values.append(np.array(z['RhoBumpUpDown'][ircurvename])/10000)
        if 'CVA_Rho' in z:
            column_names.append('CVA_Rho')
            mas_of_column_values.append(np.array(z['CVA_Rho'][ircurvename])/10000)
        if 'DVA_Rho' in z:
            column_names.append('DVA_Rho')
            mas_of_column_values.append(np.array(z['DVA_Rho'][ircurvename])/10000)
        if 'CVA_Rho' in z and 'DVA_Rho' in z:
            column_names.append('BCVA_Rho')
            mas_of_column_values.append(np.array(z['CVA_Rho'][ircurvename])/10000 + np.array(z['DVA_Rho'][ircurvename])/10000  )

    else: #это батч - считаем для PV,CVA,DVA,BCVA
        for el in r:
            el.Queries=["Rho",'CVA_Rho','DVA_Rho']
        r=ps.get(ps.put(r))
        z=ps.make_result_id(algo_id, r._id)
        try:
            ps_utils.compute(ps.put(r),algo_id=algo_id)
        except:
            print("Batch has some failed cpties")
        dates=list(map(lambda x: x.date(),r[0].Model.MarketDataSet.RatesCurvesBundles[ircurvename].RatesSchedule.Dates))
        tenors=list(map(lambda x: (x-datetime.datetime.now().date()).days/365, dates))
        column_names=['Dates','Tenors','PV_Rho','CVA_Rho','DVA_Rho','BCVA_Rho']
        mas_of_column_values=[dates,tenors]
        mas_pv_dv01=np.zeros(len(dates))
        mas_cva_dv01=np.zeros(len(dates))
        mas_dva_dv01=np.zeros(len(dates))
        mas_bcva_dv01=np.zeros(len(dates))
        for el in ps.get(z).Result:
            try:
                s=el.StatusText
            except:
                s=None
            if s=='done':
                mas_pv_dv01+=np.array(el.Result['Rho'][ircurvename])/10000
                try:
                    mas_cva_dv01+=np.array(el.Result['CVA_Rho'][ircurvename])/10000
                    mas_dva_dv01+=np.array(el.Result['DVA_Rho'][ircurvename])/10000
                    mas_bcva_dv01+=(np.array(el.Result['CVA_Rho'][ircurvename])/10000+np.array(el.Result['DVA_Rho'][ircurvename])/10000)
                except:
                    pass
        mas_of_column_values.append(mas_pv_dv01)
        mas_of_column_values.append(mas_cva_dv01)
        mas_of_column_values.append(mas_dva_dv01)
        mas_of_column_values.append(mas_bcva_dv01)
    df=pd.DataFrame({a:b for (a,b) in zip(column_names, mas_of_column_values)})
    totals=['Total','Total']
    totals+=list(map(lambda x: sum(x), [df[col_name] for col_name in df.columns[2:]]))
    df=df.append({a:b for (a,b) in zip(df.columns, totals)},ignore_index=True)
    return df


def get_tenored_vega(r,ircurvename, algo_id, use_aad=True, use_bump=False, use_bump_updown=False,
                     only_pv_greeks=False, is_batch=False):
    if ircurvename not in ['RUB_KEY_RATE','RUB_RUONIA_OIS']:
            raise Exception("Only RUB_KEY_RATE and RUB_RUONIA_OIS Vega are supported")
    r=ps.get(r) 
    if not is_batch:
        if r.Model.TypeName in ['Cheyette1FactorModelMC','HybridCheyette1FactorMCLocalVolatilityModel']:
            if 'NettingSets' in r.Product:
                r.Product=r.Product.NettingSets[0].MarginSets[0].Products[0]
            r.Model.LocalVolatilityMinimum=1e-13
            r.Model.CalibrationFrequencyInDays=30
            r.Model.LocalVolNumberOfTimeStepsPerYear=250
            r.Model.MonteCarloMaxStepInHours=24
        if use_aad:
            r.Queries=["InterestRatesVega",'CVA_InterestRatesVega','DVA_InterestRatesVega']
        if only_pv_greeks:
            r.Queries.remove('CVA_InterestRatesVega')
            r.Queries.remove('DVA_InterestRatesVega')
        if use_bump:
            r.Queries.append('VegaBump')
        if use_bump_updown:
            r.Queries.append('VegaBumpUpDown')
        z=ps_utils.compute(ps.put(r),algo_id=algo_id)
        tenors=r.Model.MarketDataSet.VolatilitySurfaces[ircurvename].Tenors
        column_names=['Tenors']
        mas_of_column_values=[tenors]
        if 'InterestRatesVega' in z:
            column_names.append('PV_Vega')
            mas_of_column_values.append(np.array(z['InterestRatesVega'][ircurvename]).flatten().reshape(len(tenors),-1).sum(axis=1)/10000)
        if 'VegaBump' in z:
            column_names.append('PV_VegaBump')
            mas_of_column_values.append(np.array(z['VegaBump'][ircurvename]).flatten().reshape(len(tenors),-1).sum(axis=1)/10000)
        if 'VegaBumpUpDown' in z:
            column_names.append('PV_VegaBumpUpDown')
            mas_of_column_values.append(np.array(z['VegaBumpUpDown'][ircurvename]).flatten().reshape(len(tenors),-1).sum(axis=1)/10000)
        if 'CVA_InterestRatesVega' in z:
            column_names.append('CVA_Vega')
            mas_of_column_values.append(np.array(z['CVA_InterestRatesVega'][ircurvename]).flatten().reshape(len(tenors),-1).sum(axis=1)/10000)
        if 'DVA_InterestRatesVega' in z:
            column_names.append('DVA_Vega')
            mas_of_column_values.append(np.array(z['DVA_InterestRatesVega'][ircurvename]).flatten().reshape(len(tenors),-1).sum(axis=1)/10000)
        if 'CVA_InterestRatesVega' in z and 'DVA_InterestRatesVega' in z:
            column_names.append('BCVA_Vega')
            mas_of_column_values.append(np.array(z['CVA_InterestRatesVega'][ircurvename]).flatten().reshape(len(tenors),-1).sum(axis=1)/10000 \
                                        + np.array(z['DVA_InterestRatesVega'][ircurvename]).flatten().reshape(len(tenors),-1).sum(axis=1)/10000  )
    else: #это батч - считаем для PV,CVA,DVA,BCVA
        for el in r:
            el.Queries=["InterestRatesVega",'CVA_InterestRatesVega','DVA_InterestRatesVega']
        r=ps.get(ps.put(r))
        z=ps.make_result_id(algo_id, r._id)
        try:
            ps_utils.compute(ps.put(r),algo_id=algo_id)
        except:
            print("Batch has some failed cpties")
        tenors=r[0].Model.MarketDataSet.VolatilitySurfaces[ircurvename].Tenors
        column_names=['Tenors','PV_InterestRatesVega','CVA_InterestRatesVega','DVA_InterestRatesVega','BCVA_InterestRatesVega']
        mas_of_column_values=[tenors]
        mas_pv_vega=np.zeros(len(tenors))
        mas_cva_vega=np.zeros(len(tenors))
        mas_dva_vega=np.zeros(len(tenors))
        mas_bcva_vega=np.zeros(len(tenors))
        for el in ps.get(z).Result:
            try:
                s=el.StatusText
            except:
                s=None
            if s=='done':
                mas_pv_vega+=np.array(el.Result['InterestRatesVega'][ircurvename]).flatten().reshape(len(tenors),-1).sum(axis=1)/10000
                try:
                    mas_cva_vega+=np.array(el.Result['CVA_InterestRatesVega'][ircurvename]).flatten().reshape(len(tenors),-1).sum(axis=1)/10000
                    mas_dva_vega+=np.array(el.Result['DVA_InterestRatesVega'][ircurvename]).flatten().reshape(len(tenors),-1).sum(axis=1)/10000
                    mas_bcva_vega+=np.array(el.Result['CVA_InterestRatesVega'][ircurvename]).flatten().reshape(len(tenors),-1).sum(axis=1)/10000
                    mas_bcva_vega+=np.array(el.Result['DVA_InterestRatesVega'][ircurvename]).flatten().reshape(len(tenors),-1).sum(axis=1)/10000
                
                except:
                    pass
        mas_of_column_values.append(mas_pv_vega)
        mas_of_column_values.append(mas_cva_vega)
        mas_of_column_values.append(mas_dva_vega)
        mas_of_column_values.append(mas_bcva_vega)
    
    df=pd.DataFrame({a:b for (a,b) in zip(column_names, mas_of_column_values)})
    totals=['Total']
    totals+=list(map(lambda x: sum(x), [df[col_name] for col_name in df.columns[1:]]))
    df=df.append({a:b for (a,b) in zip(df.columns, totals)},ignore_index=True)
    return df
    
    
def get_matrix_vega(r,ircurvename, metric_name, algo_id, is_batch=False):
    if ircurvename not in ['RUB_KEY_RATE','RUB_RUONIA_OIS']:
            raise Exception("Only RUB_KEY_RATE and RUB_RUONIA_OIS DV01 are supported")
    r=ps.get(r) 
    metric_name+='_InterestRatesVega'
    if metric_name=='PV_InterestRatesVega':
        metric_name='InterestRatesVega'
    if not is_batch:
        raise Exeption('Only bucketed requsets are supported!')
    else:
        for el in r:
                el.Queries=['InterestRatesVega','CVA_InterestRatesVega','DVA_InterestRatesVega']
        r=ps.get(ps.put(r))
        z=ps.make_result_id(algo_id, r._id)
        try:
            ps_utils.compute(ps.put(r),algo_id=algo_id)
        except:
            print("Batch has some failed cpties")
        tenors=r[0].Model.MarketDataSet.VolatilitySurfaces[ircurvename].Tenors
        strikes=r[0].Model.MarketDataSet.VolatilitySurfaces[ircurvename].Strikes
        column_names=['Tenors']+[el for el in strikes]
        mas_of_column_values=[tenors]
        matrix=np.zeros(len(tenors)*len(strikes)).reshape(len(tenors),-1)
        for el in ps.get(z).Result:
            try:
                s=el.StatusText
            except:
                s=None
            if s=='done':
                try:
                    matrix+=np.array(el.Result[metric_name][ircurvename]).flatten().reshape(len(tenors),-1)/10000
                except:
                    if metric_name=='BCVA_InterestRatesVega':
                        try:
                            matrix+=np.array(el.Result['CVA_InterestRatesVega'][ircurvename]).flatten().reshape(len(tenors),-1)/10000
                            matrix+=np.array(el.Result['DVA_InterestRatesVega'][ircurvename]).flatten().reshape(len(tenors),-1)/10000
                        except:
                            pass
        for i in range(len(strikes)):
            mas_of_column_values.append(matrix[:,i])
        df=pd.DataFrame({a:b for (a,b) in zip(column_names, mas_of_column_values)})
        totals=['Total']
        totals+=list(map(lambda x: sum(x), [df[col_name] for col_name in df.columns[1:]]))
        df=df.append({a:b for (a,b) in zip(df.columns, totals)},ignore_index=True)
        df=df.set_index('Tenors')
        return df
    
    



    
def get_mds_with_shifted_ir_or_credit_curve(r, ircurvename=None, creditcurvename=None, shift_in_bp=None, level_in_bp=None):
    r=ps.get(r) 
    if type(r)==ps.message.MessageList:
        mds=r[0].Model.MarketDataSet
    elif type(r)==ps.message.GlobalID:
        mds=r.Model.MarketDataSet
    else:
        raise Exception('Unsupported type of r!!!!')
        
    if shift_in_bp is not None:
        if ircurvename is not None:
            mds.RatesCurvesBundles[ircurvename].RatesSchedule.Rates = [x+0.01*shift_in_bp for x in \
                                                    mds.RatesCurvesBundles[ircurvename].RatesSchedule.Rates]
        elif creditcurvename is not None:
            mds.CreditCurves[creditcurvename].RatesSchedule.Rates = [ x + 0.01*shift_in_bp/0.75 for x in\
                                                        mds.CreditCurves[creditcurvename].RatesSchedule.Rates]
            mds.CreditCurves[creditcurvename].Quotes.Values = [ x*0.75 for x in\
                                                  mds.CreditCurves[creditcurvename].RatesSchedule.Rates]
    elif level_in_bp is not None:
        if ircurvename is not None:
            mds.RatesCurvesBundles[ircurvename].RatesSchedule.Rates = [ 0.01*level_in_bp for x in \
                                                    mds.RatesCurvesBundles[ircurvename].RatesSchedule.Rates]
        elif creditcurvename is not None:
            mds.CreditCurves[creditcurvename].RatesSchedule.Rates = [0.01*level_in_bp/0.75 for x in\
                                                        mds.CreditCurves[creditcurvename].RatesSchedule.Rates]
            mds.CreditCurves[creditcurvename].Quotes.Values = [ x*0.75 for x in\
                                                  mds.CreditCurves[creditcurvename].RatesSchedule.Rates]
    mds=ps.put(mds)
    return mds

In [ ]:
#how to get dv01 of a batch request

r=ps.get('dae32fc68cf183bdb1da561f3439be0724b3d8d8')
 
#r.Model.TypeName=no_skew_model
 
df=get_tenored_dv01(r,'RUB_KEY_RATE',algo_id='4b6e60a41ed694ddf8b13153d2bffc38ec6fcad0',\
                    use_aad=True, use_bump=False, use_bump_updown=False, is_batch=True)
 
df

In [11]:
ps_utils.switch_to_dev()
mas=[]
r=ps.get('cfbfe0a30787626087d50657dc93caefc37ea370')
for el in r.Result:
    if(ps.get(el._nonce)['_Counterparty'] in ['FOTON']):
        print(ps.get(el._nonce)['_Counterparty'],el._id)
        break
        #mas.append(ps.get(el._nonce)._id)
#ps.put(ps.new_cm(mas))

[INFO] 2023-10-06 11:24:36,541 - Nothing to do: current environment is already dev


In [ ]:
r=ps.get('df264acf68e68c5e648f0b86ed20721009294b89')
ps_utils.compute(ps.put(r),algo_id='e4fe902c3d2e3233571dc28142c37e87a9554d7b')
#ps.compute_raw('e4fe902c3d2e3233571dc28142c37e87a9554d7b',r)

In [ ]:
#wanted=['MKSMO','FRTGS','GRIDD','SIBRH','ROSNB','SZPPB','POLU','SZKOL','PRMMH','MBPGL','PROOM']
wanted=['AMRZT','UELMD','ALBNR','URBEN','POKRR','SZARR','BAVER' 'NERUS', 'LUMYE']
mas=[]
r=ps.get('8c2bd60d5535ec1663d0478f6398c57f208ee2b7')
for el in r.Result:
    if(ps.get(el._nonce)['_Counterparty'] in wanted ):
        mas.append(ps.get(el._nonce)._id)
r=ps.get(ps.put(ps.new_cm(mas)))
for el in r:
    el.Queries=['ProductValue','BCVA']
z=ps_utils.compute(ps.put(r))
for i,el in enumerate(z):
    print(el['ProductValue'],el['BCVA'],ps.get(r[i]._id)['_Counterparty'])

In [ ]:
ps_utils.switch_to_dev()
#mas=[]
r=ps.get('034733b0f1e050103cb78ca36d448a41b3fe8be9')
for el in r.Result:
    try:
        s=el.StatusText
    except:
        s='None'
    if (s!='done'):
        #print(el._id,s[0:65])
        #print(ps.get(el._nonce)['_Counterparty'], el._id,s[0:65])
        print(ps.get(el._nonce)['_Counterparty'], ps.get(el._nonce).Product.CounterpartyCreditRating, el._id,s[0:65])
        #mas.append(ps.get(el._nonce)._id)
#ps_utils.compute(ps.put(ps.new_cm(mas)), algo_id='1879d93d4ae4cfddc51b58f0218426de4e6f5da8')
#ps.put(ps.new_cm(mas))

In [ ]:
import ps, ps_utils
#_from = 'psi'; _to = 'prod_pvlss'
_from = 'prod_pvlss'; _to = 'dev'
_id = '5187db871f5bc97976c35ff44250b15a9333746e'

ps_utils.upload_message(_id, _from, _to)

ps_utils.switch_to_env(_to)
print(f"check result")
n, not_found = ps_utils.chk_msg_tree(ps.get(_id))
print(f"\ndone: {n}, not found: {not_found}\n")
if not_found:
    print(f"copy missing messages")
    for m_id in not_found:
        ps_utils.upload_message(m_id, _from, _to)
    print(f"check result")
    n, not_found = ps_utils.chk_msg_tree(ps.get(_id))
    print(f"\ndone: {n}, not found: {not_found}\n")


In [ ]:
#Крос 
ps_utils.switch_to_prod_pvlss()

print("Product number",1)
r=ps_utils.pricing.XVA(ps.get('88ddc65ff27a820e07ebb8d3442e727d2298f9ec')[0])
r.Product.NettingSets[0].MarginSets[0].Products=[r.Product.NettingSets[0].MarginSets[0].Products[2]]


r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])


r.Model.MarketDataSet.RatesCurvesBundles['RUB_KEY_RATE'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['EUR_XCCY'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['CNH_FX'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['RUB_SOFR'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['USD_SOFR'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['RUB_RUONIA_OIS'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['EUR_EURIBOR_6M'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['EUR_RUB'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['USD_SOFR'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['USD_LIBOR_3M'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['USD_XCCY'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['XAU_USD'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['CNH_SOFR'].InterpolationMethod='Linear'


new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.8, 'RUB_CHMF_OFZ_SNR_CR': 6.00}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)



print(r)

z=r.price(algo_id='e4fe902c3d2e3233571dc28142c37e87a9554d7b')
bcva=abs(z['BCVA'])
vega=abs((z['CVA_ParallelForeignExchangeVega']['EURRUB']/100 + z['DVA_ParallelForeignExchangeVega']['EURRUB']/100))+abs((z['CVA_ParallelForeignExchangeVega']['USDRUB']/100 + z['DVA_ParallelForeignExchangeVega']['USDRUB']/100))
delta=abs((z['CVA_CurrencyExposure']['EUR'] + z['DVA_CurrencyExposure']['EUR']))
rho=abs((z['CVA_ParallelRho']['EUR_XCCY']/10000 + z['DVA_ParallelRho']['EUR_XCCY']/10000))
bcva+=vega*3
bcva+=delta*0.025
bcva+=rho*50*2
bpv=117268
rate=r.Product.NettingSets[0].MarginSets[0].Products[0].Leg2FixedRates[0]
rate_new=rate+bcva/bpv/10000
print("bcva with greeks charge=",bcva, "bcva all-in in bp=",bcva/bpv, "vega=",vega, "delta=",delta, "rho",rho*2, "rate_old=",rate,"rate_new",rate_new)

for i in range (1):
    r.Product.NettingSets[0].MarginSets[0].Products[0].Leg2FixedRates=[rate_new for i in range(len(r.Product.NettingSets[0].MarginSets[0].Products[0].Leg2FixedRates))]
    z=r.price(algo_id='e4fe902c3d2e3233571dc28142c37e87a9554d7b')
    bcva=abs(z['BCVA'])
    vega=abs((z['CVA_ParallelForeignExchangeVega']['EURRUB']/100 + z['DVA_ParallelForeignExchangeVega']['EURRUB']/100))+abs((z['CVA_ParallelForeignExchangeVega']['USDRUB']/100 + z['DVA_ParallelForeignExchangeVega']['USDRUB']/100))
    delta=abs((z['CVA_CurrencyExposure']['EUR'] + z['DVA_CurrencyExposure']['EUR']))
    rho=abs((z['CVA_ParallelRho']['EUR_XCCY']/10000 + z['DVA_ParallelRho']['EUR_XCCY']/10000))
    bcva+=vega*3
    bcva+=delta*0.025
    bcva+=rho*50*2
    bpv=117268
    rate=r.Product.NettingSets[0].MarginSets[0].Products[0].Leg2FixedRates[0]
    rate_new=rate+bcva/bpv/10000
    print("iteration=", i+1, "bcva with greeks charge=",bcva, "bcva all-in in bp=",bcva/bpv, "vega=",vega, "delta=",delta, "rho",rho*2, "rate_old=",rate,"rate_new",rate_new)



In [ ]:
#форвард 
r=ps_utils.pricing.XVA(ps.get('3c317cbab3a4b26e0d6ad7b01263ca00c9618cd8'))


r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])




new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.8, 'RUB_CHMF_OFZ_SNR_CR': 6.00}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)
    
r.Product.NettingSets[0].MarginSets[0].Products=[r.Product.NettingSets[0].MarginSets[0].Products[0]]

z=r.price(algo_id='e444e5c23f460ab1a8bb318afb4416c1d0573cf1')
bcva=abs(z['BCVA'])
vega=abs((z['CVA_ParallelForeignExchangeVega']['CNHRUB']/100 + z['DVA_ParallelForeignExchangeVega']['CNHRUB']/100))
delta=abs((z['CVA_CurrencyExposure']['CNH']/100 + z['DVA_CurrencyExposure']['CNH']/100))
bcva+=vega*3
bcva+=delta*r.Model.MarketDataSet.Spots['USDCNH']*0.02/r.Model.MarketDataSet.Spots['USDRUB']
N=abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)
strike=r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
strike_new=(1+bcva/N)*strike
print("bcva=",bcva, "vega=",vega, "delta=",delta, "strike_old=",strike,"strike_new",strike_new)

r.Product.NettingSets[0].MarginSets[0].Products[0].Strike=strike_new
z=r.price(algo_id='e444e5c23f460ab1a8bb318afb4416c1d0573cf1')
bcva=abs(z['BCVA'])
vega=abs((z['CVA_ParallelForeignExchangeVega']['CNHRUB']/100 + z['DVA_ParallelForeignExchangeVega']['CNHRUB']/100))
delta=abs((z['CVA_CurrencyExposure']['CNH']/100 + z['DVA_CurrencyExposure']['CNH']/100))
bcva+=vega*3
bcva+=delta*r.Model.MarketDataSet.Spots['USDCNH']*0.02/r.Model.MarketDataSet.Spots['USDRUB']
N=abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)
strike=r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
print("bcva=",bcva, "vega=",vega, "delta=",delta, "strike_all_in=",strike, "bcva_all_in=",bcva)

#get_bcva_df(z)


In [ ]:
#make pnl explain request
ps_utils.switch_to_dev()
import ps, ps_utils

layer2_algo_id = 'dbb2472abfde197ecd5e14b030fdc84207a5b92c'

layer1_algo_id = 'e4fe902c3d2e3233571dc28142c37e87a9554d7b'

prev_response = '61597888dd3187725035150670042f746a7e033b'
response = '1a333661f0b02b4d2ce9e989961563ffda9f3332'

request = ps.new_cm({

    'RequestName':  'PortfolioPnLExplain',

    'PricingAlgoID': layer1_algo_id,

    'ReportingCurrency': 'USD',

    '_Method': 'Revaluation',

    'Queries': ['BCVA', 'CVA', 'DVA'],

    'PreviousResponse': prev_response,

    'Response': response,

})

ps.put(request)

print("Request=", request._id)

ps_utils.compute(request, algo_id=layer2_algo_id)

# 01-11-2023 среда

In [ ]:
# даша яблокова AEDRUB 

In [24]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('127affe9a1f686c1a3ff89b98bcbf2dcec9f2a2c')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

# r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
# r.Model.LocalVolatilityMinimum=1e-13
# r.Model.CalibrationFrequencyInDays=30
# r.Model.LocalVolNumberOfTimeStepsPerYear=250
# r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)
#r.StaticDataSet='961c90aa9d7f9fedc21b12a42ff53ab29d107d4c'
    
r.Model.CalibrationUnderlyings['AED']='AED_FX'  
r.Model.MarketDataSet.VolatilitySurfaces['AED_FX']=r.Model.MarketDataSet.VolatilitySurfaces['CNH_FX']
r.set_correlation('AEDRUB','USDRUB',0.6)
r.set_correlation('AEDRUB','AED_FX',0.0)
r.set_correlation('AED_FX','USDRUB',0.0)
r.set_correlation('AEDRUB','RUB_KEY_RATE',0.0)
r.set_correlation('AED_FX','RUB_KEY_RATE',0.0)
r.set_correlation('AEDRUB','USD_LIBOR_3M',0.0)
r.set_correlation('AED_FX','USD_LIBOR_3M',0.0)


    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-02 11:28:08,234 - Nothing to do: current environment is already dev
[INFO] 2023-11-02 11:28:11,754 - Compute:839147e01274dfabce9c036138a56e2176c607b4->1193585894cf99fb4c3f1a811724631cee40f638


Dict([('BCVA', -134312.09112523877),
      ('LegBCVA',
       [-336.148658943853,
        -16179.46009892193,
        -2772.3010205693813,
        -55690.851275418616,
        -2772.3010205693813,
        -47577.73974200466,
        -336.148658943853,
        -14611.929810274078]),
      ('Legs',
       [170e68fc3f1386c6e101da425489b494345b141a,
        8db1e36e0e37613143b3c971d960a6e300e65eef,
        19fec62bbea142a5c325df8222833f7384c5d528,
        18d60a1bbc07762ddf098ed1143511691e6c045e,
        8b098d8ef4c331fb378cbea78bebf529b8e9dfbc,
        e3db75fc494f832e8b9c91f276e26a01c407387c,
        4b977420e29d86dc3c694fdfe09641f060497e95,
        04856c02d7716429a7e2b34e6269678b5c80d93b]),
      ('CVA', -148753.2541682156),
      ('DVA', 14441.16304297682),
      ('LegPresentValues',
       [33521.003841435304,
        2027855.1250902712,
        102945.02614385495,
        3522095.5789550394,
        102945.02614385495,
        2156130.152210012,
        33521.003841435304,
        1

# Тая PRMMH restruct 9 рейтинг

In [3]:
(215113-187693)*1.05*7162088

28791.0

In [4]:
(204899-187693)*1.05

18066.3

In [5]:
(215133-187693)*1.05

28812.0

In [7]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('b6c780a9fc68e8c141f8769bc082c1e648725f42')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)



    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-01 15:27:36,597 - Nothing to do: current environment is already dev
[INFO] 2023-11-01 15:27:44,791 - Compute:0bd4dd57c6fd8e991860ee19636b2ead02916ba7->c6473e04349a52d8952e888004621080d919e880


Dict([('BCVA', -879770.9806473947),
      ('LegBCVA',
       [-200241.74103374415,
        -230109.30052178176,
        -219218.1009774479,
        -230201.83811442074]),
      ('Legs',
       [a312dd8845242de3d0cecfe360d55766681f4bab,
        9b25bafef1b252efcbd1036f4e433afbe60a9954,
        b663d0a63d02764a85cd1e6d25f27276d57e2557,
        63fbc361e1b29452516aae5552cd48f85cd2d081]),
      ('CVA', -879972.2959730863),
      ('DVA', 201.31532569157793),
      ('LegPresentValues',
       [5340502.181594783,
        6895335.52240276,
        6766345.66131072,
        6929940.809816146]),
      ('ProductValue', 25932124.175124414),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 49713.244084295446),
      ('CashToSettleToday', 0.0)])

In [9]:
(230109-200241)*1.05

31361.4

In [10]:
(219218-200241)*1.05

19925.850000000002

In [12]:
(230201-200241)*1.05

31458.0

In [45]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('b6c780a9fc68e8c141f8769bc082c1e648725f42')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)
mas=r.Product.NettingSets[0].MarginSets[0].Products



prem=mas[1].RollingPremiums
newprem=[]
for i in range(len(prem)):
    if (i>=8 and i<=22):
        newprem.append(4.614/4.6*prem[i])
    else:
        newprem.append(prem[i])

r.Product.NettingSets[0].MarginSets[0].Products=[mas[1]]
r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=newprem
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-01 16:07:26,694 - Nothing to do: current environment is already dev
[INFO] 2023-11-01 16:07:27,975 - Compute:7629a113805681aacbd67bcf0f7602bd0cc5644b->46d8ffd420491ac3d116324e7b19bf276b30d1c3


Dict([('BCVA', -230370.54280733285),
      ('LegBCVA', [-230370.54280733285]),
      ('Legs', [f9911da6cf3d6798aa9c6be9c64288ca4e2b4895]),
      ('CVA', -230420.87163875575),
      ('DVA', 50.32883142289448),
      ('LegPresentValues', [6927611.992422452]),
      ('ProductValue', 6927611.992422452),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 13129.440481306594),
      ('CashToSettleToday', 0.0)])

In [46]:
6927611-6895335

32276

In [43]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('b6c780a9fc68e8c141f8769bc082c1e648725f42')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)
mas=r.Product.NettingSets[0].MarginSets[0].Products

prem=mas[2].RollingPremiums

newprem=[]
for i in range(len(prem)):
    if (i>=8 and i<=22):
        newprem.append(5.34/5.33*prem[i])
    else:
        newprem.append(prem[i])

r.Product.NettingSets[0].MarginSets[0].Products=[mas[2]]
r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=newprem
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-01 16:06:48,239 - Nothing to do: current environment is already dev
[INFO] 2023-11-01 16:06:49,457 - Compute:30563d73bdcc103e21135a37d0b0f7a04030a6b6->fb1e3fcdac658cd58635ead11be40efcbc3d4081


Dict([('BCVA', -219367.72166955768),
      ('LegBCVA', [-219367.72166955768]),
      ('Legs', [86874255a33f08fe414a6c4481e6f9b598f72b52]),
      ('CVA', -219418.05050098058),
      ('DVA', 50.32883142289448),
      ('LegPresentValues', [6786487.94839298]),
      ('ProductValue', 6786487.94839298),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 12963.155793840655),
      ('CashToSettleToday', 0.0)])

In [34]:
(219442-200241)

19201

In [44]:
6786487-6766345

20142

In [56]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('b6c780a9fc68e8c141f8769bc082c1e648725f42')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)
mas=r.Product.NettingSets[0].MarginSets[0].Products

prem=mas[3].RollingPremiums

newprem=[]
for i in range(len(prem)):
    if (i>=8 and i<=22):
        newprem.append(4.214/4.2*prem[i])
    else:
        newprem.append(prem[i])

r.Product.NettingSets[0].MarginSets[0].Products=[mas[3]]
r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=newprem
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-01 16:12:02,811 - Nothing to do: current environment is already dev
[INFO] 2023-11-01 16:12:03,976 - Compute:8ee603b78b8bd2ed09d2307c4509e30c08563e0e->0cb0f34bbfcad7b4597f7b8f061aa7cff8669dab


Dict([('BCVA', -230464.56366796244),
      ('LegBCVA', [-230464.56366796244]),
      ('Legs', [72b6b9dd41c009a81712cfcc6a6f4e5892470f87]),
      ('CVA', -230514.89249938534),
      ('DVA', 50.32883142289448),
      ('LegPresentValues', [6962310.656646999]),
      ('ProductValue', 6962310.656646999),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 12959.02242764012),
      ('CashToSettleToday', 0.0)])

In [57]:
6962310-6929940

32370

In [60]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('702c8b76e5ced8d2777bec8ae2484383e0aa860f')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)

# mas=r.Product.NettingSets[0].MarginSets[0].Products

# prem=mas[3].RollingPremiums

# newprem=[]
# for i in range(len(prem)):
#     if (i>=8 and i<=22):
#         newprem.append(4.214/4.2*prem[i])
#     else:
#         newprem.append(prem[i])

# r.Product.NettingSets[0].MarginSets[0].Products=[mas[3]]
# r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=newprem
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-01 16:16:38,649 - Nothing to do: current environment is already dev
[INFO] 2023-11-01 16:16:40,052 - Compute:4efcd9389cd25af467cd0e11b53e29c9f0d957bf->19a9472c6fc347d22b7e1c9be705e14250c1d74b


Dict([('BCVA', -1577813.564961714),
      ('CVA', -1577814.4352816243),
      ('DVA', 0.8703199103126104),
      ('ProductValue', 6668859.465981768),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 11289.216261431147),
      ('CashToSettleToday', 0.0)])

In [ ]:
3086266

In [61]:
z=r.run_xva_breakdown(show=True)

[INFO] 2023-11-01 16:18:27,122 - Compute:b8622046359ae9b9c23469df03192cb21494fd71->4d0358cafac5bf3f140996a2d539cc420c6d5def


# 02-11-2023 четверг 

# Паша HIMPP 15 рейтинг менем барьер в будущем с 1.49 на 6.07 без cva-aal-in

In [4]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('1a4922bfb4665f46cc3cce27f533e4b6843e9fbd')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)

mas=r.Product.NettingSets[0].MarginSets[0].Products

# prem=mas[3].RollingPremiums

# newprem=[]
# for i in range(len(prem)):
#     if (i>=8 and i<=22):
#         newprem.append(4.214/4.2*prem[i])
#     else:
#         newprem.append(prem[i])

r.Product.NettingSets[0].MarginSets[0].Products=[mas[0]]
# r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=newprem
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-02 11:15:26,778 - Nothing to do: current environment is already dev
[INFO] 2023-11-02 11:15:29,618 - Compute:8559d064baacc1bc3fae29a02c274a1180c3dee5->7c2d3fb34161214dcb11c39240a232875bdb0221


Dict([('BCVA', -199.3825747572276),
      ('LegBCVA', [-199.3825747572276]),
      ('Legs', [a3a8134303503198293a51b38c4369035307431b]),
      ('CVA', -231.54189783137866),
      ('DVA', 32.15932307415108),
      ('LegPresentValues', [23290.033617141024]),
      ('ProductValue', 23290.033617141024),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 56.224080453421),
      ('CashToSettleToday', 0.0)])

In [14]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('1a4922bfb4665f46cc3cce27f533e4b6843e9fbd')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)

mas=r.Product.NettingSets[0].MarginSets[0].Products

prem=mas[1].RollingPremiums

newprem=[]
for i in range(len(prem)):
    if (i>=9):
        newprem.append(6.13/6.07*prem[i])
    else:
        newprem.append(prem[i])

r.Product.NettingSets[0].MarginSets[0].Products=[mas[1]]
r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=newprem
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-02 11:19:39,131 - Nothing to do: current environment is already dev
[INFO] 2023-11-02 11:19:40,300 - Compute:04c303f9aef82745c2d31dc796e35a3f1e83b9b4->cf078d8a7597a0de5c19fa735fc6f59ce8419f5c


Dict([('BCVA', -1749.806621595292),
      ('LegBCVA', [-1749.806621595292]),
      ('Legs', [6fe34ff407fabdcd1180e66f6c8c97f34bf26ef8]),
      ('CVA', -1749.82400662765),
      ('DVA', 0.01738503235806632),
      ('LegPresentValues', [43049.8196707491]),
      ('ProductValue', 43049.8196707491),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 114.02575568437308),
      ('CashToSettleToday', 0.0)])

In [8]:
(1969-199)

1770

In [15]:
43049-41240

1809

# Женя Васин норникель 13 рейтинг крос AEDRUB

In [29]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('4ba9639b3e3606a309808bb2e6031e3dd3f196c3')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

# r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
# r.Model.LocalVolatilityMinimum=1e-13
# r.Model.CalibrationFrequencyInDays=30
# r.Model.LocalVolNumberOfTimeStepsPerYear=250
# r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)
r.Model.CalibrationUnderlyings['AED']='AED_FX'  

    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-02 14:05:32,458 - Nothing to do: current environment is already dev
[INFO] 2023-11-02 14:05:33,989 - Compute:d16c3205a8e1d7e1842395f2ea62b6007896f86c->fd30168e5b4f5eda224b0b3dc83a392ef1d0d672


Dict([('BCVA', -87573.21470555157),
      ('CVA', -102876.30652836987),
      ('DVA', 15303.091822818293),
      ('ProductValue', 1952895.1945933849),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 43688.44200850977),
      ('CashToSettleToday', 0.0)])

In [30]:
z=r.run_xva_breakdown(show=True)

[INFO] 2023-11-02 14:06:36,448 - Compute:9cac1a417d6deb874be1e7d1955c0d3a07126fb1->e18bb63c592a99b62d0e58f42cd0415bc30b0282


In [31]:
#6

ps_utils.switch_to_dev()
           
r=ps_utils.pricing.XVA(ps.get('4ba9639b3e3606a309808bb2e6031e3dd3f196c3')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
# r.set_queries(["ProductValue","ParallelInterestRatesVega","ParallelForeignExchangeVega",
#                "ParallelCreditDelta","ParallelRho","CurrencyExposure"])
r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])


#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

# r.Product.pop('CounterpartyCreditRating')
# r.Product.CounterpartyCreditCurveIdentifier='RUB_CHMF_OFZ_SNR_CR'

# new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.3}
# for cr_curve, cr_value in new_credit_values.items():
#     r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
#                                                                        len(r.Model.MarketDataSet.CreditCurves[
#                                                                                cr_curve].RatesSchedule.Rates)

#r.Product.NettingSets[0].MarginSets[0].Products.pop(0)

r.Model.CalibrationUnderlyings['AED']='AED_FX'  

z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')
get_bcva_df(z)


[INFO] 2023-11-02 14:07:33,555 - Nothing to do: current environment is already dev
[INFO] 2023-11-02 14:07:34,615 - Compute:a34f3bc59bcb2dd808a8b948a6571259df9deca7->f23fba965f9c7a877bd36f03a47cced2906b476d


PV= 1950022.1279349178
Unexpected item: ProductValue
Unexpected item: ReportingCurrency
Unexpected item: StandardError
Unexpected item: CashToSettleToday
Unexpected item: BCVA


CVA         DVA  \
VA                          VA                     -97,799.71   26,028.49   
ParallelInterestRatesVega   AED_FX                      -0.15        0.18   
                            RUB_KEY_RATE                -1.90        1.20   
ParallelForeignExchangeVega AEDRUB                  -3,853.96    2,381.20   
ParallelCreditDelta         RUB_MINFIN_OFZ            -495.43        0.00   
                            RUB_SBER_OFZ_SNR_CR          0.00      215.61   
ParallelRho                 AED_FX                      96.36       30.49   
                            RUB_KEY_RATE                -0.75        0.47   
                            RUB_RUONIA_OIS               9.32       -2.13   
                            RUB_SOFR                   -96.30      -30.53   
                            USD_SOFR                    -0.00        0.00   
CurrencyExposure            AED                 -1,087,366.53 -344,711.05   
                            RUB                    989,566.82  370,739.54   
                            USD                         -0.00        0.00   

                                                         BCVA  
VA                          VA                     -71,771.22  
ParallelInterestRatesVega   AED_FX                       0.03  
                            RUB_KEY_RATE                -0.70  
ParallelForeignExchangeVega AEDRUB                  -1,472.76  
ParallelCreditDelta         RUB_MINFIN_OFZ            -495.43  
                            RUB_SBER_OFZ_SNR_CR        215.61  
ParallelRho                 AED_FX                     126.85  
                            RUB_KEY_RATE                -0.28  
                            RUB_RUONIA_OIS               7.19  
                            RUB_SOFR                  -126.83  
                            USD_SOFR                    -0.00  
CurrencyExposure            AED                 -1,432,077.57  
                            RUB                  1,360,306.35  
                            USD                         -0.00

In [35]:
bcva=87573 + 1432077*0.02 + 495*7 + 1472*3
bcva/8087

15.345064919005813

# Паша AAARU 11 рейтинг рестракт 

In [37]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('a807738ad849563a1cd60ba0176528213d386066')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)

#mas=r.Product.NettingSets[0].MarginSets[0].Products

# prem=mas[3].RollingPremiums

# newprem=[]
# for i in range(len(prem)):
#     if (i>=8 and i<=22):
#         newprem.append(4.214/4.2*prem[i])
#     else:
#         newprem.append(prem[i])

#r.Product.NettingSets[0].MarginSets[0].Products=[mas[0]]
# r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=newprem
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-02 15:11:36,643 - Nothing to do: current environment is already dev
[INFO] 2023-11-02 15:11:43,510 - Compute:fa5a66c4299cbf870c0c763e144a5ef9bcc4c296->415b48eaf86d1388ffbf81865bcabb43680f3546


Dict([('BCVA', -15.930227351342525),
      ('LegBCVA', [0.7101488211198017, -32.1438853656397]),
      ('Legs',
       [867d39624db1a2485169c454c3338d5c1e386ab1,
        8a4e464ae2d843d205210fc5dbe99f9adfa7a0d4]),
      ('CVA', -23.704523126902746),
      ('DVA', 7.774295775560221),
      ('LegPresentValues', [-5296.315740351745, -1254.1856069746623]),
      ('ProductValue', -6550.501347326403),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 24.106961453124132),
      ('CashToSettleToday', 0.0)])

In [50]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('66458eef68a69a1991a3267e565d051653b46017'))
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)

mas=r.Product.NettingSets[0].MarginSets[0].Products

prem=mas[0].RollingPremiums
barriers=mas[0].UpBarrierLevels

newprem=[]
newbarriers=[]
for i in range(len(prem)):
    if (i>=3):
        newprem.append(3.72/1.35*prem[i])
        newbarriers.append(0.2025)
    else:
        newprem.append(prem[i])
        newbarriers.append(barriers[i])

#r.Product.NettingSets[0].MarginSets[0].Products=[mas[0]]
print(prem)
print(newprem)

r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=newprem
r.Product.NettingSets[0].MarginSets[0].Products[0].UpBarrierLevels=newbarriers
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-02 15:39:06,935 - Nothing to do: current environment is already dev


[138698.63013698, 221917.808219185, 229315.068493152, 221917.808219185, 228809.791152029, 228688.52459017, 29508.1967212951, 161372.95081966, 200102.459016399, 32274.5901639165, 138319.672131137, 27663.9344262142, 119877.049180317, 27663.9344262419, 66393.4426229473, 13831.9672131071, 47950.8196721268, 7377.0491803238]
[138698.63013698, 221917.808219185, 229315.068493152, 611506.8493150875, 630498.0911744799, 630163.9344262462, 81311.47540979093, 444672.1311475076, 551393.4426229661, 88934.4262294588, 381147.5409835775, 76229.50819667913, 330327.8688524291, 76229.50819675546, 182950.81967212146, 38114.75409833957, 132131.14754097164, 20327.868852447806]


[INFO] 2023-11-02 15:39:07,224 - Compute:9b08d3d967cc6caff0ac134088254b8e94b12290->bb2b48c3d67ffa45d8043eb304dcb538c435d98d


Dict([('BCVA', -30.67480173827159),
      ('LegBCVA', [-30.67480173827159]),
      ('Legs', [cafc2261d6b199472547896c43f89653f0c822dd]),
      ('CVA', -39.167403164936246),
      ('DVA', 8.492601426664654),
      ('LegPresentValues', [-2513.3496646632316]),
      ('ProductValue', -2513.3496646632316),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 33.42579652264224),
      ('CashToSettleToday', 0.0)])

# Паша INNNS restruct 9 рейтинг 

In [52]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('570e5463cc919c461b2adf96445ff0fc249a3f87')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

# new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
# for cr_curve, cr_value in new_credit_values.items():
#     r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
#                                                                        len(r.Model.MarketDataSet.CreditCurves[
#                                                                                cr_curve].RatesSchedule.Rates)

# mas=r.Product.NettingSets[0].MarginSets[0].Products

# prem=mas[0].RollingPremiums
# barriers=mas[0].UpBarrierLevels

# newprem=[]
# newbarriers=[]
# for i in range(len(prem)):
#     if (i>=3):
#         newprem.append(3.72/1.35*prem[i])
#         newbarriers.append(0.2025)
#     else:
#         newprem.append(prem[i])
#         newbarriers.append(barriers[i])

# #r.Product.NettingSets[0].MarginSets[0].Products=[mas[0]]
# print(prem)
# print(newprem)

# r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=newprem
# r.Product.NettingSets[0].MarginSets[0].Products[0].UpBarrierLevels=newbarriers
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-02 18:17:58,682 - Nothing to do: current environment is already dev
[INFO] 2023-11-02 18:17:59,662 - Compute:f7ef6418340efe0409e649137c8bbdcf89b99965->1359a96e2b87935c069a11ed161e2155b683bfb8


Dict([('BCVA', -1099.3387490083387),
      ('LegBCVA', [-567.056969680707, -532.2949121279298]),
      ('Legs',
       [021ce60d85266e29e638ba0169089c5ce857b1ab,
        399b78750d56f9552952cecee1cdb11b0688e955]),
      ('CVA', -15092.050837205683),
      ('DVA', 13992.712088197344),
      ('LegPresentValues', [-205770.61362198202, -206934.4653866034]),
      ('ProductValue', -412705.0790085858),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 4258.446577170228),
      ('CashToSettleToday', 0.0)])

# 03-11-2023 пятница 

# Паша SINDE 13 рейтинг снижение номинала 

In [53]:
#как есть 

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('07bc3827e892234ff24f9f97210cc3d5afd0d66f')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)

# mas=r.Product.NettingSets[0].MarginSets[0].Products

# prem=mas[0].RollingPremiums
# barriers=mas[0].UpBarrierLevels

# newprem=[]
# newbarriers=[]
# for i in range(len(prem)):
#     if (i>=3):
#         newprem.append(3.72/1.35*prem[i])
#         newbarriers.append(0.2025)
#     else:
#         newprem.append(prem[i])
#         newbarriers.append(barriers[i])

# #r.Product.NettingSets[0].MarginSets[0].Products=[mas[0]]
# print(prem)
# print(newprem)

# r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=newprem
# r.Product.NettingSets[0].MarginSets[0].Products[0].UpBarrierLevels=newbarriers
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-03 10:59:03,779 - Nothing to do: current environment is already dev
[INFO] 2023-11-03 10:59:07,543 - Compute:5a374ee10ff8b8b170f6c70c5efeb0eabd540389->df4a380dc9d2ce067250d3783887ebcad3fad219


Dict([('BCVA', -61.918744613946764),
      ('LegBCVA',
       [-4.2046274298462425,
        -36.58172046561855,
        -2.1967750420815495,
        -19.11270661373298]),
      ('Legs',
       [aeae4ceb0eb39200416385e62822e51fdfbdf587,
        a393b5f3f7b852d0035548f15908f51410fc9487,
        4f8248505eecb63100b99a3a8cd6fae3ac64fb96,
        5277d77a398b7b304d8d582397a0301e681f2265]),
      ('CVA', -61.9194681612989),
      ('DVA', 0.0007235473521397763),
      ('LegPresentValues',
       [457.70917822415413,
        4366.243451678456,
        239.13750165723422,
        2281.2139239980097]),
      ('ProductValue', 7344.30405555787),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 10.451835606048059),
      ('CashToSettleToday', 0.0)])

# Тема MBKRK 14 рейтинг рестракт 

In [14]:
#355 млн в конце  

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('5a2ec0449626c3972352d73f44487207ed13a6fb')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)

# mas=r.Product.NettingSets[0].MarginSets[0].Products

# prem=mas[0].RollingPremiums
# barriers=mas[0].UpBarrierLevels

# newprem=[]
# newbarriers=[]
# for i in range(len(prem)):
#     if (i>=3):
#         newprem.append(3.72/1.35*prem[i])
#         newbarriers.append(0.2025)
#     else:
#         newprem.append(prem[i])
#         newbarriers.append(barriers[i])

# #r.Product.NettingSets[0].MarginSets[0].Products=[mas[0]]
# print(prem)
# print(newprem)

# r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=newprem
# r.Product.NettingSets[0].MarginSets[0].Products[0].UpBarrierLevels=newbarriers

#r.Product.NettingSets.pop(1)

r.Product.NettingSets[0].MarginSets[0].Products.append('56b18499f7462711d9dfdc8beca6ebb4c37138cc')
r.Product.NettingSets[0].MarginSets[0].Products.append('be0a6ce5f69a7480b1087fb71311425b43d03570')

r.Product.NettingSets[0].MarginSets[0].Products[0].UseRateAveragingApproximation=False
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-03 12:26:42,306 - Nothing to do: current environment is already dev
[INFO] 2023-11-03 12:26:44,264 - Compute:8747ab0c3b9e4b4c0a4ecbcdc63178821336a174->b4f86116b2e5e67c4148a51d6f13f3d3c8c00a2a


Dict([('BCVA', -576556.0241429259),
      ('LegBCVA',
       [-247276.57551643963,
        -79827.25297110374,
        -243998.98101283575,
        -5453.214642546737]),
      ('Legs',
       [5d62c4674683df9275b2381ea39c8bc4357885ab,
        692856b9ae76ebd02adfd09572b6aa306cf5f702,
        56b18499f7462711d9dfdc8beca6ebb4c37138cc,
        be0a6ce5f69a7480b1087fb71311425b43d03570]),
      ('CVA', -576556.0241429259),
      ('DVA', 0.0),
      ('LegPresentValues',
       [3018487.663489634,
        917729.8267396316,
        2287776.9687345563,
        52378.78818304688]),
      ('ProductValue', 6276373.247146872),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 10027.650234673369),
      ('CashToSettleToday', 0.0)])

In [15]:
charge=(79827+243998+5453-247276)
print(charge)
charge*r.Model.MarketDataSet.Spots['USDRUB']

82002


7609375.59

In [32]:
#385 млн в конце  

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('5a2ec0449626c3972352d73f44487207ed13a6fb')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)

# mas=r.Product.NettingSets[0].MarginSets[0].Products

# prem=mas[0].RollingPremiums
# barriers=mas[0].UpBarrierLevels

# newprem=[]
# newbarriers=[]
# for i in range(len(prem)):
#     if (i>=3):
#         newprem.append(3.72/1.35*prem[i])
#         newbarriers.append(0.2025)
#     else:
#         newprem.append(prem[i])
#         newbarriers.append(barriers[i])

# #r.Product.NettingSets[0].MarginSets[0].Products=[mas[0]]
# print(prem)
# print(newprem)

# r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=newprem
# r.Product.NettingSets[0].MarginSets[0].Products[0].UpBarrierLevels=newbarriers

#r.Product.NettingSets.pop(1)

r.Product.NettingSets[0].MarginSets[0].Products.append('d569b9141b48c9fd99ca5a2b8855d4d3710a756e')
r.Product.NettingSets[0].MarginSets[0].Products.append('692c479545b5b895ff7923954b85a417da1c7249')

r.Product.NettingSets[0].MarginSets[0].Products[0].UseRateAveragingApproximation=False
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-03 14:12:51,049 - Nothing to do: current environment is already dev
[INFO] 2023-11-03 14:12:52,121 - Compute:95554b7bd0524483bee554e414197378c05b16e0->1dd1e052339a63c5a8ce51e143375b4d066447ec


Dict([('BCVA', -615714.5634915319),
      ('LegBCVA',
       [-247276.57551643963,
        -79827.25297110374,
        -280416.73937296047,
        -8193.995631028063]),
      ('Legs',
       [5d62c4674683df9275b2381ea39c8bc4357885ab,
        692856b9ae76ebd02adfd09572b6aa306cf5f702,
        d569b9141b48c9fd99ca5a2b8855d4d3710a756e,
        692c479545b5b895ff7923954b85a417da1c7249]),
      ('CVA', -615714.5634915319),
      ('DVA', 0.0),
      ('LegPresentValues',
       [3018487.663489634,
        917729.8267396316,
        2629236.217799181,
        76828.3310411017]),
      ('ProductValue', 6642282.03906955),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 10044.124288600393),
      ('CashToSettleToday', 0.0)])

In [27]:
charge=(79827+280416+8193-247276)
print(charge)
charge*r.Model.MarketDataSet.Spots['USDRUB']

121160


11243042.200000001

In [29]:
247276-79827

167449

In [31]:
267967.6325866554-86236

181731.63258665538

# Аня крос Илим 12 рейтинг 

In [57]:
#385 млн в конце  

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('2ec95ccf10367b541b775e38bfdc3889474aa89f')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

# r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
# r.Model.LocalVolatilityMinimum=1e-13
# r.Model.CalibrationFrequencyInDays=30
# r.Model.LocalVolNumberOfTimeStepsPerYear=250
# r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

r.Model.MarketDataSet.RatesCurvesBundles['RUB_KEY_RATE'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['EUR_XCCY'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['CNH_FX'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['RUB_SOFR'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['USD_SOFR'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['RUB_RUONIA_OIS'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['EUR_EURIBOR_6M'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['EUR_RUB'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['USD_SOFR'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['USD_LIBOR_3M'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['USD_XCCY'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['XAU_USD'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['CNH_SOFR'].InterpolationMethod='Linear'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)


#r.Product.NettingSets[0].MarginSets[0].Products.pop(0)
# r.Product.NettingSets[0].MarginSets[0].Products[0].UseRateAveragingApproximation=False

#r.Product.NettingSets[0].MarginSets[0].Products=['88bd8d2a539e599b36f0a26554a5f144f4317036']
    
#z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')
z=r.price(algo_id='94b781627f848a9bf567caea0521d1472b29d80a')


z




[INFO] 2023-11-03 16:05:02,804 - Nothing to do: current environment is already dev
[INFO] 2023-11-03 16:05:03,884 - Compute:5c3c547a0965c0587f8722b32dc2e396b31946a4->537df96ba81940dae0284d356acb0ff69c0094d5


Dict([('BCVA', -1578.174616297158),
      ('LegBCVA', [-86.40894128439682, -1554.9120231782967]),
      ('Legs',
       [75c362aaa508c82cf3b6e633ab91f6343e500155,
        111827db32716a4a2f57973df204ae848d5efe15]),
      ('CVA', -1962.9653770321204),
      ('DVA', 384.7907607349623),
      ('LegPresentValues', [-205.07825273051822, 197261.999726385]),
      ('ProductValue', 197056.92147365457),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 519.9329273615009),
      ('CashToSettleToday', 0.0)])

In [58]:
z=r.run_xva_breakdown(show=True,algo_id='94b781627f848a9bf567caea0521d1472b29d80a')

[INFO] 2023-11-03 16:05:28,842 - Compute:3ff29be6e2d147cf7e1db24246504ea6cff592f7->81061e4193cb4bbcd120ad8180d064f2cbb7edb9


In [51]:
r.get_cashflows(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

[INFO] 2023-11-03 15:16:32,321 - Compute:0ad09f28528e53830f2b472c84bc491ccef6cbac->2671780c074046f3cb23a43e114825b9d23a8794


,Description,Amount,StandardError,Currency,Leg,StatusText
Date,,,,,,
2025-01-30,Leg1FixedCoupon,"27,625.61",1.55,RUB,0,Estimate
2025-01-30,Leg2FixedCoupon,"-10,153.40",5.69,EUR,0,Estimate
2025-01-30,Leg1FinalNotionalExchangePayment,"756,768.15",42.45,RUB,0,Estimate
2025-01-30,Leg2FinalNotionalExchangePayment,"-846,856.45",474.66,EUR,0,Estimate


In [53]:
#6

ps_utils.switch_to_dev()
           
r=ps_utils.pricing.XVA(ps.get('2ec95ccf10367b541b775e38bfdc3889474aa89f')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
# r.set_queries(["ProductValue","ParallelInterestRatesVega","ParallelForeignExchangeVega",
#                "ParallelCreditDelta","ParallelRho","CurrencyExposure"])
r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])


#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

# r.Product.pop('CounterpartyCreditRating')
# r.Product.CounterpartyCreditCurveIdentifier='RUB_CHMF_OFZ_SNR_CR'

r.Model.MarketDataSet.RatesCurvesBundles['RUB_KEY_RATE'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['EUR_XCCY'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['CNH_FX'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['RUB_SOFR'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['USD_SOFR'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['RUB_RUONIA_OIS'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['EUR_EURIBOR_6M'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['EUR_RUB'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['USD_SOFR'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['USD_LIBOR_3M'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['USD_XCCY'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['XAU_USD'].InterpolationMethod='Linear'
r.Model.MarketDataSet.RatesCurvesBundles['CNH_SOFR'].InterpolationMethod='Linear'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)


z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')
get_bcva_df(z)


[INFO] 2023-11-03 15:17:39,362 - Nothing to do: current environment is already dev
[INFO] 2023-11-03 15:17:39,961 - Compute:21104ccc39d65bf450084c204451350442317c15->f0a7f9a0865f8af6a3e234032842693024967dfd


PV= 196652.58871773642
Unexpected item: ProductValue
Unexpected item: ReportingCurrency
Unexpected item: StandardError
Unexpected item: CashToSettleToday
Unexpected item: BCVA


CVA       DVA  \
VA                          VA                   -1,955.40    382.36   
ParallelInterestRatesVega   EUR_EURIBOR_6M           -0.00      0.00   
                            RUB_KEY_RATE             -0.07      0.03   
ParallelForeignExchangeVega EURRUB                  -46.38     18.23   
ParallelCreditDelta         RUB_MINFIN_OFZ          -10.80      0.00   
                            RUB_SBER_OFZ_SNR_CR       0.00      5.41   
ParallelRho                 EUR_EURIBOR_3M            2.66      0.26   
                            EUR_EURIBOR_6M           -0.00      0.00   
                            EUR_RUB                  -0.00      0.00   
                            EUR_XCCY                 -1.49     -0.36   
                            RUB_KEY_RATE             -0.02      0.01   
                            RUB_RUONIA_OIS            0.11     -0.05   
                            RUB_SOFR                  1.49      0.36   
                            USD_SOFR                  0.00      0.00   
CurrencyExposure            EUR                  12,653.74  2,945.52   
                            RUB                 -14,609.13 -2,563.16   
                            USD                       0.00     -0.00   

                                                      BCVA  
VA                          VA                   -1,573.03  
ParallelInterestRatesVega   EUR_EURIBOR_6M           -0.00  
                            RUB_KEY_RATE             -0.04  
ParallelForeignExchangeVega EURRUB                  -28.15  
ParallelCreditDelta         RUB_MINFIN_OFZ          -10.80  
                            RUB_SBER_OFZ_SNR_CR       5.41  
ParallelRho                 EUR_EURIBOR_3M            2.92  
                            EUR_EURIBOR_6M           -0.00  
                            EUR_RUB                  -0.00  
                            EUR_XCCY                 -1.86  
                            RUB_KEY_RATE             -0.01  
                            RUB_RUONIA_OIS            0.05  
                            RUB_SOFR                  1.86  
                            USD_SOFR                  0.00  
CurrencyExposure            EUR                  15,599.26  
                            RUB                 -17,172.29  
                            USD                       0.00

In [55]:
bcva=1573*1.02 + 15599*0.02 + 10*7  + 28*3
bcva/301

6.878538205980067

In [38]:
bcva

76264.68

In [39]:
14.55+1.53

16.080000000000002

In [44]:
Nrub=np.array(r.Product.NettingSets[0].MarginSets[0].Products[1].Leg1Notionals)
Neur=np.array(r.Product.NettingSets[0].MarginSets[0].Products[1].Leg2Notionals)
Nrub/Neur

array([99.29, 99.29, 99.29, 99.29, 99.29])

# Тема PVBMV гарантор OZONN 10 рейтинга рестракт 

In [59]:
#385 млн в конце  

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('d4c92ab4b61bb099ee3c8eb04f6da7d593684265')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'



new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)


#r.Product.NettingSets[0].MarginSets[0].Products.pop(0)
# r.Product.NettingSets[0].MarginSets[0].Products[0].UseRateAveragingApproximation=False

#r.Product.NettingSets[0].MarginSets[0].Products=['88bd8d2a539e599b36f0a26554a5f144f4317036']
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')
#z=r.price(algo_id='94b781627f848a9bf567caea0521d1472b29d80a')

z




[INFO] 2023-11-03 16:13:27,484 - Nothing to do: current environment is already dev
[INFO] 2023-11-03 16:13:33,526 - Compute:04c9231a2d54b2302296b1e9753daa3b680baac3->b63a0e1d0919f41b28766b649f02967d50a6cd4e


Dict([('BCVA', -52482.36018486954),
      ('LegBCVA', [-24806.754775353, -27843.02566773729]),
      ('Legs',
       [ec90de0a98494f8fc12d8e93888f13c72f8b79c5,
        23a8afaf6c0c97779688f04da54d85eb228697d8]),
      ('CVA', -59416.09567047322),
      ('DVA', 6933.7354856036845),
      ('LegPresentValues', [-454882.86606623064, -90129.37631424384]),
      ('ProductValue', -545012.2423804745),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 9680.386832944325),
      ('CashToSettleToday', 0.0)])

In [60]:
(27843-24806)

3037

# Саша Сорокин Норникель 

In [77]:
#385 млн в конце  

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('75aeb72ed71a9b4d7fc64371dce777bb72b3dd9a')[0])
r.Queries.append('ExpectedCounterpartyProbabilityOfDefault')
# r.Queries.append('LegPresentValues')

# r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
# r.Model.LocalVolatilityMinimum=1e-13
# r.Model.CalibrationFrequencyInDays=30
# r.Model.LocalVolNumberOfTimeStepsPerYear=250
# r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'



new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)


#r.Product.NettingSets[0].MarginSets[0].Products.pop(0)
# r.Product.NettingSets[0].MarginSets[0].Products[0].UseRateAveragingApproximation=False

r.Product.NettingSets[0].MarginSets[0].Products=[r.Product.NettingSets[0].MarginSets[0].Products[-1]]
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')

z




[INFO] 2023-11-03 16:55:28,859 - Nothing to do: current environment is already dev
[INFO] 2023-11-03 16:55:30,121 - Compute:5ddc887b766734954e4d8c8c6a8367b485aeec3f->1c68bf4b1f2cef59ce82bd84cc0b9e332fd61de7


Dict([('BCVA', -46505.8248897654),
      ('LegBCVA', [-46505.8248897654]),
      ('Legs', [254bdee0b6327387e64e99ce7db4c074ec3b0679]),
      ('CVA', -78416.03355049607),
      ('DVA', 31910.208660730666),
      ('LegPresentValues', [-675017.5911881179]),
      ('ProductValue', -675017.5911881179),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 47072.222452055415),
      ('CashToSettleToday', 0.0),
      ('ExpectedCounterpartyProbabilityOfDefault',
       Dict([('Dates',
              [datetime.datetime(2023, 11, 6, 0, 0, tzinfo=datetime.timezone.utc),
               datetime.datetime(2023, 11, 13, 0, 0, tzinfo=datetime.timezone.utc),
               datetime.datetime(2023, 11, 20, 0, 0, tzinfo=datetime.timezone.utc),
               datetime.datetime(2023, 11, 27, 0, 0, tzinfo=datetime.timezone.utc),
               datetime.datetime(2023, 12, 4, 0, 0, tzinfo=datetime.timezone.utc),
               datetime.datetime(2023, 12, 11, 0, 0, tzinfo=datetime.timezone.utc),
          

In [79]:
sum(z['ExpectedCounterpartyProbabilityOfDefault']['Values'])

0.027327423382559446

In [71]:
z=r.run_xva_breakdown(show=True)

[INFO] 2023-11-03 16:49:54,916 - Compute:a3d3f25b85771d2b51ff79d935834bfe236ac5e8->72c2c3920e443c3650d04c4aa26339a307a40a6e


In [61]:
#6

ps_utils.switch_to_dev()
           
r=ps_utils.pricing.XVA(ps.get('75aeb72ed71a9b4d7fc64371dce777bb72b3dd9a')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
# r.set_queries(["ProductValue","ParallelInterestRatesVega","ParallelForeignExchangeVega",
#                "ParallelCreditDelta","ParallelRho","CurrencyExposure"])
r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])


#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'

# r.Product.pop('CounterpartyCreditRating')
# r.Product.CounterpartyCreditCurveIdentifier='RUB_CHMF_OFZ_SNR_CR'



new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)


z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')
get_bcva_df(z)


[INFO] 2023-11-03 16:21:25,115 - Nothing to do: current environment is already dev
[INFO] 2023-11-03 16:21:28,406 - Compute:9fd94cce6695440b71814c6d6d724fa72c05f32f->8707b46f15a45f61abdb62fda5c72448f773a65f


PV= 855942.8670006841
Unexpected item: ProductValue
Unexpected item: ReportingCurrency
Unexpected item: StandardError
Unexpected item: CashToSettleToday
Unexpected item: BCVA


CVA         DVA  \
VA                          VA                    -130,004.62   48,364.57   
ParallelInterestRatesVega   RUB_KEY_RATE                -3.01        1.10   
                            USD_LIBOR_3M                -2.22        0.86   
ParallelForeignExchangeVega USDRUB                  -8,433.27    2,869.44   
ParallelCreditDelta         RUB_MINFIN_OFZ            -626.14        0.00   
                            RUB_SBER_OFZ_SNR_CR          0.00      687.82   
ParallelRho                 RUB_KEY_RATE                -1.16        0.43   
                            RUB_RUONIA_OIS               9.83       -3.88   
                            RUB_SOFR                   124.99       48.42   
                            USD_LIBOR_3M                -0.60        0.23   
                            USD_SOFR                  -124.88      -48.46   
                            USD_XCCY                     0.00        0.00   
CurrencyExposure            RUB                 -2,064,115.81 -675,359.02   
                            USD                  1,934,111.19  723,723.59   

                                                         BCVA  
VA                          VA                     -81,640.05  
ParallelInterestRatesVega   RUB_KEY_RATE                -1.92  
                            USD_LIBOR_3M                -1.37  
ParallelForeignExchangeVega USDRUB                  -5,563.83  
ParallelCreditDelta         RUB_MINFIN_OFZ            -626.14  
                            RUB_SBER_OFZ_SNR_CR        687.82  
ParallelRho                 RUB_KEY_RATE                -0.73  
                            RUB_RUONIA_OIS               5.94  
                            RUB_SOFR                   173.41  
                            USD_LIBOR_3M                -0.37  
                            USD_SOFR                  -173.34  
                            USD_XCCY                     0.00  
CurrencyExposure            RUB                 -2,739,474.83  
                            USD                  2,657,834.78

In [67]:
#форвард 

for i in range(5):
    r=ps_utils.pricing.XVA(ps.get('75aeb72ed71a9b4d7fc64371dce777bb72b3dd9a')[0])


    r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])




    new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
    for cr_curve, cr_value in new_credit_values.items():
        r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                           len(r.Model.MarketDataSet.CreditCurves[
                                                                                   cr_curve].RatesSchedule.Rates)


    r.Product.NettingSets[0].MarginSets[0].Products=[r.Product.NettingSets[0].MarginSets[0].Products[i]]

    z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')
    bcva=abs(z['BCVA'])
    vega=abs((z['CVA_ParallelForeignExchangeVega']['USDRUB']/100 + z['DVA_ParallelForeignExchangeVega']['USDRUB']/100))
    delta=abs((z['CVA_CurrencyExposure']['USD']/100 + z['DVA_CurrencyExposure']['USD']/100))
    bcva+=vega*3
    bcva+=delta*0.02
    N=abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)
    strike=r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
    strike_new=(1+bcva/N)*strike
    print("i=", i+1, "bcva=",bcva, "vega=",vega, "delta=",delta, "strike_old=",strike,"strike_new",strike_new)

    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike=strike_new
    z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')
    bcva=abs(z['BCVA'])
    vega=abs((z['CVA_ParallelForeignExchangeVega']['USDRUB']/100 + z['DVA_ParallelForeignExchangeVega']['USDRUB']/100))
    delta=abs((z['CVA_CurrencyExposure']['USD']/100 + z['DVA_CurrencyExposure']['USD']/100))
    bcva+=vega*3
    bcva+=delta*0.02
    N=abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)
    strike=r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
    print("i=", i+1, "bcva=",bcva, "vega=",vega, "delta=",delta, "strike_all_in=",strike, "bcva_all_in=",bcva)

#get_bcva_df(z)


[INFO] 2023-11-03 16:40:57,900 - Compute:21989940ffc331df83b29d572f5b3a571f2b69d9->00ab350c86de4625594f715a7253882572d863a9


i= 1 bcva= 2414.4914029490337 vega= 100.19645995694722 delta= 1351.476093052751 strike_old= 94.1316 strike_new 94.13387279938947


[INFO] 2023-11-03 16:40:58,646 - Compute:58eb0955d374c440d6ffef5d80cbdb5010e9b29c->c9cbe8fd4b98df43d60595a9a2096a8f92dcb53d


i= 1 bcva= 2417.7757618975147 vega= 100.1830873668099 delta= 1351.9932075583347 strike_all_in= 94.13387279938947 bcva_all_in= 2417.7757618975147


[INFO] 2023-11-03 16:40:59,381 - Compute:4c3556449643b531113ee97f051697c4d3dc042c->3a35d198ca12839fc027d4a6836671f8472e6b2c


...

i= 4 bcva= 20017.494152244704 vega= 1264.3791754070146 delta= 6426.840321490696 strike_all_in= 96.66671846869718 bcva_all_in= 20017.494152244704


[INFO] 2023-11-03 16:41:08,294 - Compute:ba31519cfc9a9ca18837e513c8c7b199b07f81e1->9d152692e6ac08ad2f789e47eb9b05d7f0373ce5


i= 5 bcva= 56745.908369100835 vega= 3369.755672493876 delta= 12249.55639329009 strike_old= 99.6961 strike_new 99.75267345755357


[INFO] 2023-11-03 16:41:21,319 - Compute:9ab6bcf022fc31b669fc11c9273db2141e799675->bf99571bde941d1ad41d429812d4dff429b992be


i= 5 bcva= 57476.32667310063 vega= 3372.208536857994 delta= 12283.158813220844 strike_all_in= 99.75267345755357 bcva_all_in= 57476.32667310063


In [68]:
57476/1e08

0.00057476

In [87]:
57476/1e08

0.00057476

In [94]:
sum(z['ExpectedCounterpartyProbabilityOfDefault']['Values'])

0.025970112781847132

In [95]:
lambd=(130+75)/10000/0.75
T=1
PD=(1-np.exp(-lambd*T))
0.75*PD*6965440

140857.6952450807

In [96]:
PD

0.026963158153221367

# Аня Кокс 13 рейтинг форвард USDRUB

In [103]:
#385 млн в конце  

ps_utils.switch_to_dev()         
r=ps_utils.pricing.XVA(ps.get('2675f5deb423805a350f4df1ea7892bf35e68682')[0])
# r.Queries.append('LegBCVA')
# r.Queries.append('LegPresentValues')

# r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
# r.Model.LocalVolatilityMinimum=1e-13
# r.Model.CalibrationFrequencyInDays=30
# r.Model.LocalVolNumberOfTimeStepsPerYear=250
# r.Model.MonteCarloMaxStepInHours=24

# r.NumberOfPaths=700000
# r.Model.AmericanMonteCarloNumberOfPaths=2000

#r.Model.TypeName='HybridCheyette1FactorANLocalVolatilityModel'



new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)


#r.Product.NettingSets[0].MarginSets[0].Products.pop(0)
# r.Product.NettingSets[0].MarginSets[0].Products[0].UseRateAveragingApproximation=False

#r.Product.NettingSets[0].MarginSets[0].Products=['88bd8d2a539e599b36f0a26554a5f144f4317036']

r.Model.MarketDataSet.Spots['USDRUB']=92.41875
    
z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')
#z=r.price(algo_id='94b781627f848a9bf567caea0521d1472b29d80a')

z




[INFO] 2023-11-03 17:40:40,152 - Nothing to do: current environment is already dev
[INFO] 2023-11-03 17:40:43,755 - Compute:1ede386a4dbd3dbdd59b0677fa8475272c297901->cff8df0209d929cfafc243b9df13786ffa13eef2


Dict([('BCVA', -12194.366799510026),
      ('CVA', -16451.95920050524),
      ('DVA', 4257.592400995214),
      ('ProductValue', 1459051.4781501503),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 54207.66117392441),
      ('CashToSettleToday', 0.0)])

In [104]:
z=r.run_xva_breakdown(show=True)

[INFO] 2023-11-03 17:41:09,422 - Compute:429c9ef9bdd769d0454451ce30e9c20e1a8d8d6d->50e64c6d3833c6382f21c90c1ff9ebbc507114d4


In [106]:
#форвард 

for i in range(1):
    r=ps_utils.pricing.XVA(ps.get('2675f5deb423805a350f4df1ea7892bf35e68682')[0])


    r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])




    new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.7, 'RUB_MINFIN_OFZ': 1.30}
    for cr_curve, cr_value in new_credit_values.items():
        r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                           len(r.Model.MarketDataSet.CreditCurves[
                                                                                   cr_curve].RatesSchedule.Rates)

    r.Model.MarketDataSet.Spots['USDRUB']=92.41875
    r.Product.NettingSets[0].MarginSets[0].Products=[r.Product.NettingSets[0].MarginSets[0].Products[i]]

    z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')
    bcva=abs(z['BCVA'])
    vega=abs((z['CVA_ParallelForeignExchangeVega']['USDRUB']/100 + z['DVA_ParallelForeignExchangeVega']['USDRUB']/100))
    delta=abs((z['CVA_CurrencyExposure']['USD']/100 + z['DVA_CurrencyExposure']['USD']/100))
    bcva+=vega*3
    bcva+=delta*0.02
    N=abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)
    strike=r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
    strike_new=(1+bcva/N)*strike
    print("i=", i+1, "bcva=",bcva, "vega=",vega, "delta=",delta, "strike_old=",strike,"strike_new",strike_new)

    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike=strike_new
    z=r.price(algo_id='0534e2d4a32283b9777c8d6a640bd1c1c9470ec7')
    bcva=abs(z['BCVA'])
    vega=abs((z['CVA_ParallelForeignExchangeVega']['USDRUB']/100 + z['DVA_ParallelForeignExchangeVega']['USDRUB']/100))
    delta=abs((z['CVA_CurrencyExposure']['USD']/100 + z['DVA_CurrencyExposure']['USD']/100))
    bcva+=vega*3
    bcva+=delta*0.02
    N=abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)
    strike=r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
    print("i=", i+1, "bcva=",bcva, "vega=",vega, "delta=",delta, "strike_all_in=",strike, "bcva_all_in=",bcva)

#get_bcva_df(z)


[INFO] 2023-11-03 17:42:20,235 - Compute:75518f08488393070f7929673be29950719bcb6b->e65f1f9637f025d8616727dc2f9d5d42f4e04c96


i= 1 bcva= 14267.406230950623 vega= 612.4694880929335 delta= 6732.048689254245 strike_old= 94.2819 strike_new 94.28574330905006


[INFO] 2023-11-03 17:42:27,215 - Compute:45a4f577ccb0a0c1fc7638a8410fb7be62e7c388->22c802986778ce2dad858d9c2ca03e414dcad477


i= 1 bcva= 14295.192974475303 vega= 612.3938613751795 delta= 6735.216821382615 strike_all_in= 94.28574330905006 bcva_all_in= 14295.192974475303


In [107]:
((1+14295/3.5e8)*94.2819 - 94.2819)*100

0.38507421728439795

In [110]:
r.Model.MarketDataSet.RatesCurvesBundles['CNH_SOFR']

CMSG<e2af08051ca3e30c0f6052d8f69d6ab4ccd90868>{TypeName:'DiscountFactorCurve',Identifier:'CNH_SOFR',Currency:'CNH',InterpolationDayCountConvention:'Act365',InterpolationMethod:'CubicSpline',ExtrapolationMethod:'Flat',InterpolationSpaceCoordinates:'Rates',RateQuoteExpression:'Percentage',RatesSchedule:{Dates:[2023-11-06 00:00:00+00:00,2023-11-07 00:00:00+00:00,2023-11-14 00:00:00+00:00,2023-11-21 00:00:00+00:00,2023-11-28 00:00:00+00:00,2023-12-07 00:00:00+00:00,2024-01-08 00:00:00+00:00,2024-02-07 00:00:00+00:00,2024-03-07 00:00:00+00:00,2024-04-08 00:00:00+00:00,2024-05-07 00:00:00+00:00,2024-06-07 00:00:00+00:00,2024-07-08 00:00:00+00:00,2024-08-07 00:00:00+00:00,2024-09-09 00:00:00+00:00,2024-10-07 00:00:00+00:00,2024-11-07 00:00:00+00:00,2025-05-07 00:00:00+00:00,2025-11-07 00:00:00+00:00,2026-11-09 00:00:00+00:00,2027-11-08 00:00:00+00:00,2028-11-07 00:00:00+00:00,2029-11-07 00:00:00+00:00,2030-11-07 00:00:00+00:00,2031-11-07 00:00:00+00:00,2032-11-08 00:00:00+00:00,2033-11-07 00:

In [14]:
import pandas as pd
import numpy as np
df=pd.read_excel('ПриложениеСпецТМЦ.xlsx')
df=df.iloc[1:-1,:]
df

,№ п/п,Инв. Номер,Наименование,Кол-во,"Цена за ед. без НДС, рублей","Стоимость без НДС, рублей","НДС 20%, рублей","Стоимость с учетом НДС, рублей"
1,1.0,9.945991e+11,Ванна моечная НСО2М-12/6Б,1.0,1790.85,1790.85,358.17,2149.02
2,2.0,9.945991e+11,Ванна моечная НСО2М-12/6Б,1.0,1790.85,1790.85,358.17,2149.02
3,3.0,9.945991e+11,Ванна моечная НСО2М-12/6Б,1.0,1790.85,1790.85,358.17,2149.02
4,4.0,9.945991e+11,Ванна моечная НСО2М-12/6Б,1.0,1790.85,1790.85,358.17,2149.02
5,5.0,9.945991e+11,Ванна моечная НСО М-6/6Б,1.0,1130.85,1130.85,226.17,1357.02
...,...,...,...,...,...,...,...,...
186,186.0,1.090000e+11,Столешница топ ARTISHOK Кварцит GAYA DARK шлиф...,1.0,3573.68,3573.68,714.74,4288.42
187,187.0,1.090000e+11,Столешница топ ARTISHOK Кварцит GAYA DARK шлиф...,1.0,3573.68,3573.68,714.74,4288.42
188,188.0,1.090000e+11,Столешница топ ARTISHOK Кварцит GAYA DARK шлиф...,1.0,3573.68,3573.68,714.74,4288.42
189,189.0,1.090000e+11,Столешница топ ARTISHOK Кварцит GAYA DARK шлиф...,1.0,3573.68,3573.68,714.74,4288.42


In [24]:
df['БезНДС']=df['Кол-во']*df['Цена  за ед. без НДС,  рублей']
df['НДСнеокругл']=0.2*np.array(df['БезНДС'])
print("НДСнеокругл", sum(df['НДСнеокругл']), np.round(sum(df['НДСнеокругл']),2))

df['НДСокругл']=list(map(lambda x: np.round(x,2),df['НДСнеокругл'] ))
print("НДСокругл", sum(df['НДСокругл']),np.round(sum(df['НДСокругл']),2))

НДСнеокругл 135980.37799999982 135980.38
НДСокругл 135980.4399999998 135980.44


In [27]:
ps_utils.switch_to_dev()
r=ps.get('516478ef31f75e1feba3a5b7936a8714c5f9b74d')
ps_utils.compute(ps.put(r),algo_id='4378aad7661ecd69fcd1b35066a0aa8da52739a5')

[INFO] 2023-11-09 12:24:12,565 - Nothing to do: current environment is already dev
[INFO] 2023-11-09 12:24:12,598 - Compute:516478ef31f75e1feba3a5b7936a8714c5f9b74d->6bf03741ce3fbc48595906c6b0ff0631df5beb65


Dict([('ProductValue', -4009329.0047578486),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 2491.2372903411933),
      ('CashToSettleToday', 0.0),
      ('CVA', -0.01682563219643925),
      ('DVA', 20341.15413077466),
      ('BCVA', 20341.137305142463),
      ('CVA_Rho',
       Dict([('RUB_KEY_RATE',
              [-653623157802.8243,
               3558357992326.2666,
               -5485671374975.552,
               -1115601080712.0073,
               1498253350098.3452,
               425508382489.7253,
               -37768801011.822105,
               118672695519.28761,
               -32553667482.859333,
               4242766701.187079,
               -91573866.15796992,
               19996721.786062974,
               -5399883.528580066,
               1442619.399140029,
               -386540.9254566749,
               103544.30268667062,
               -27648.43020111806,
               7218.121260522553,
               -1568.0818938306488,
               160.5

In [ ]:
ps_utils.switch_to_prod_pvlss()
r=ps.get('516478ef31f75e1feba3a5b7936a8714c5f9b74d')
ps_utils.compute(ps.put(r),algo_id='4378aad7661ecd69fcd1b35066a0aa8da52739a5')